In [ ]:
%run "../01-check_setup.ipynb"

In [ ]:
import requests, os

os.environ["USER_AGENT"] = f"{requests.get('https://httpbin.org/user-agent').json()['user-agent']}"
os.environ["USER_AGENT"]

In [ ]:
import zipfile
import requests
from io import BytesIO

# URL of the zipped file
zip_url = "https://docs.python.org/3/archives/python-3.13-docs-text.zip"

# Download the zipped file
response = requests.get(zip_url)
zip_file = BytesIO(response.content)


In [ ]:

# Load documents from the zipped file
documents_from_zip = []
with zipfile.ZipFile(zip_file, 'r') as z:
    for file_name in z.namelist():
        if not z.getinfo(file_name).is_dir():
            with z.open(file_name) as f:
                documents_from_zip.append({'metadata': {'file_name': file_name}, 'content': f.read().decode('utf-8')})

# Check the number of documents loaded
print(f"Number of documents loaded: {len(documents_from_zip)}")

In [ ]:
## Select only records where file_name contains '/faq/'
faq_documents = [doc for doc in documents_from_zip if '/faq/' in doc['metadata']['file_name']]  
print(f"Number of FAQ documents: {len(faq_documents)}")

In [ ]:
print(faq_documents[2]['content'])

In [ ]:
import pandas as pd

## Convert faq_documents to a DataFrame `df_faq`
df_faq = pd.DataFrame(faq_documents)


In [ ]:
import markdown
from IPython.display import HTML

# Convert Markdown to HTML
html = markdown.markdown(df_faq['content'][0])

display(HTML(html))


In [ ]:
# Convert column metadata of df_faq to string
df_faq['metadata'] = df_faq['metadata'].astype(str)
df_faq['content_html'] = df_faq['content'].apply(markdown.markdown)

In [ ]:
display(df_faq.dtypes)
df_faq=df_faq.convert_dtypes()
display(df_faq.dtypes)

In [ ]:
hana_table_name="FAQ"

In [ ]:
## Create a table in SAP HANA from `df_faq`
from hana_ml.dataframe import create_dataframe_from_pandas
hdf_faq_bronze = create_dataframe_from_pandas(connection_context=myconn, 
                                              pandas_df=df_faq, 
                                              table_name=hana_table_name, 
                                              force=True,
                                              object_type_as_bin=True,
                                              table_structure={'metadata': 'NVARCHAR(5000)' ,
                                                               'content': 'NCLOB', 'content_html': 'NCLOB'}
                                              )

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 256) 

hdf_faq_bronze.head().collect().T